Softmax函数是一种常用于多分类问题的激活函数。它可以将一个包含任意实数的向量转换为概率分布形式的向量，使得每个元素的取值范围在0到1之间，并且所有元素的和为1。

Softmax函数常用于神经网络的输出层，用于表示各类别的预测概率。


这也是softmax中常见的数学性质：  
假设有三个数a, b, c，且b-a = c-b = d（等间距），  
则$$\frac{\exp(b)}{\exp(a)} = exp(b-a) = exp(d)$$  
    $$\frac{exp(c)}{exp(b)} = exp(c-b) = exp(d)$$  
所以等间距的指数比值是相等的。


In [16]:
def activation_ReLU(inputs):
    return np.maximum(0,inputs)

In [17]:
# 该函数实现了Softmax激活函数，常用于多分类神经网络的输出层。
# 输入参数inputs为二维数组（每行为一个样本，每列为一个类别的得分）。
# 1. 首先，计算每一行的最大值max_value，用于数值稳定性处理，防止指数溢出。
# 2. 用inputs减去max_value，得到slided_inputs。
# 3. 对slided_inputs进行指数运算，得到exp_values。
# 4. 对每一行的exp_values求和，得到归一化的分母norm_base。
# 5. 用exp_values除以norm_base，得到每一行的概率分布norm_values。
# 6. 返回归一化后的概率分布。

def activation_softmax(inputs):
    max_value=np.max(inputs,axis=1,keepdims=True)
    slided_inputs=inputs-max_value
    exp_values=np.exp(slided_inputs)
    norm_base=np.sum(exp_values,axis=1,keepdims=True)
    norm_values=exp_values/norm_base
    return norm_values



In [18]:
import numpy as np

# 定义softmax函数
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # 防止溢出
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# 生成2x5的矩阵
matrix = np.random.randn(2, 5)
print("原始矩阵：")
print(matrix)

# 对矩阵应用softmax
softmax_result = softmax(matrix)
print("softmax结果：")
print(softmax_result)


原始矩阵：
[[ 0.29504093  0.30678255 -0.60205285  0.85765438 -0.85088581]
 [ 0.09244222  0.22456618  0.7754452  -1.87735143 -0.37046899]]
softmax结果：
[[0.22258099 0.22520985 0.09075806 0.39068612 0.07076498]
 [0.20449906 0.2333845  0.40487013 0.02852468 0.12872163]]


### 以下是运行实现softmax的神经网络


In [19]:
#面向对象的层
class Layer:
    # 初始化函数,需要输入神经元个数和输入维度
    # self.weights和self.biases是类的属性,需要用self.来定义
    def __init__(self,n_inputs,n_neurons):
        self.weights=np.random.randn(n_inputs,n_neurons)  # weights是每个神经元的权重矩阵
        self.biases=np.random.randn(n_neurons)  # biases是每个神经元的偏置项

    # 前向传播函数,需要输入数据
    # self.sum和self.output是类的属性,需要用self.来定义
    # inputs是函数的参数,直接使用即可

    def forward(self,inputs):
        # 在Python中,当我们使用self.sum时,Python会自动为类创建一个名为sum的属性
        # 这是Python的动态特性 - 可以在运行时动态添加属性
        # 调用方法:
        # 1. 在类的其他方法中可以直接用self.sum访问
        # 2. 在类的外部可以用实例名.sum访问,比如Layer1.sum
        self.sum=np.dot(inputs,self.weights)+self.biases  # 计算加权和
        return self.sum  # 返回输出结果


In [20]:
# 下面是对上一层输出应用softmax激活函数的注释示例：
# softmax激活函数常用于多分类问题的输出层，将神经网络的输出转换为概率分布。
# 其主要思想是对每个类别的得分取指数后归一化，使所有类别的概率和为1。
# 这样可以方便地解释输出为每个类别的概率，有助于分类决策。
class Network:
    def __init__(self,network_shape):
        self.shape=network_shape
        self.layers=[]
        for i in range(len(network_shape)-1):
            layer=Layer(network_shape[i],network_shape[i+1])
            self.layers.append(layer)
    #前馈运算
    def network_forward(self,inputs):
        outputs=[inputs]
        for i in range(len(self.layers)):
            layer_output=self.layers[i].forward(outputs[i])
            if i==len(self.layers)-1:
                layer_output=activation_softmax(layer_output)
            else:
                layer_output=activation_ReLU(layer_output)
            outputs.append(layer_output)
        return outputs

In [21]:
# 测试Network类
import numpy as np

# 定义激活函数
def activation_ReLU(x):
    return np.maximum(0, x)

def activation_softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

# 创建一个简单的网络结构，例如输入3个神经元，隐藏层5个神经元，输出2个神经元
network_shape = [3, 5, 2]
net = Network(network_shape)

# 构造一个输入样本
inputs = np.array([[1.0, 2.0, 3.0]])

# 前向传播
outputs = net.network_forward(inputs)

# 打印每一层的输出
for idx, out in enumerate(outputs):
    print(f"第{idx}层输出:\n{out}\n")


第0层输出:
[[1. 2. 3.]]

第1层输出:
[[0.         3.92652249 0.34804878 5.55425097 5.60808204]]

第2层输出:
[[0.0024245 0.9975755]]

